In [1]:
import tensorflow as tf
import numpy as np

/Users/liuyanfang/anaconda3/envs/tensorflow1.4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## 保存和恢复变量

定义模型

In [2]:
graph = tf.Graph()
with graph.as_default():
    # Model parameters
    W = tf.Variable([.3], dtype=tf.float32)
    b = tf.Variable([-.3], dtype=tf.float32)
    # Model input and output
    x = tf.placeholder(tf.float32)
    linear_model = W*x + b
    y = tf.placeholder(tf.float32)
    # loss
    loss = tf.reduce_mean(tf.square(linear_model - y)) # MSE
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = optimizer.minimize(loss)

定义session, 存储变量

In [3]:
# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]

with tf.Session(graph=graph) as sess:
    saver = tf.train.Saver()
    # training loop
    init = tf.global_variables_initializer()
    sess.run(init)
    for i in range(5000):
        sess.run(train, {x: x_train, y: y_train})
    # evaluate training accuracy
    curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
    print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))
    save_path = saver.save(sess, "./tmp/model.ckpt")
    print("Model saved in file: %s" % save_path)

W: [-0.999996] b: [0.99998856] loss: 2.2052582e-11
Model saved in file: ./tmp/model.ckpt


恢复变量

In [4]:
with tf.Session(graph=graph) as sess:
    # 定义saver
    saver = tf.train.Saver()
    # 恢复模型
    saver.restore(sess, "./tmp/model.ckpt")
    curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
    print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))
    print("Model restored")

INFO:tensorflow:Restoring parameters from ./tmp/model.ckpt
W: [-0.999996] b: [0.99998856] loss: 2.2052582e-11
Model restored


选择要保存的变量

In [5]:
graph_t = tf.Graph()
with graph_t.as_default():
    a = tf.Variable(1.5, name='var_a')
    b = tf.Variable(2.5, name='var_b')
    op = a.assign_add(1)

In [6]:
with tf.Session(graph=graph_t) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    sess.run(op)
    saver = tf.train.Saver([a])  # 与 tf.train.Saver({var_a: a})等价
    saver.save(sess, './tmp/model.ckpt')

In [7]:
with tf.Session(graph=graph_t) as sess:
    # 单独初始化b
    sess.run(b.initializer)
    saver = tf.train.Saver([a])  # 与 tf.train.Saver({var_a: a})等价
    saver.restore(sess, './tmp/model.ckpt')
    print(sess.run(a))

INFO:tensorflow:Restoring parameters from ./tmp/model.ckpt
2.5


## TensorBoard

### summary

In [8]:
graph = tf.Graph()
with graph.as_default():
    # Model parameters
    W = tf.Variable([.3], dtype=tf.float32)
    b = tf.Variable([-.3], dtype=tf.float32)
    # Model input and output
    x = tf.placeholder(tf.float32)
    linear_model = W*x + b
    y = tf.placeholder(tf.float32)
    # loss
    loss = tf.reduce_mean(tf.square(linear_model - y)) # MSE
    summary_loss = tf.summary.scalar('MSE', loss)
    summary_b = tf.summary.scalar('b', b[0])
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = optimizer.minimize(loss)

    
# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]

with tf.Session(graph=graph) as sess:
    # define summary writer
    writer = tf.summary.FileWriter('./tmp/visualization/')
    # merge summaries
    merged = tf.summary.merge_all()
    # training loop
    init = tf.global_variables_initializer()
    sess.run(init)
    for i in range(5000):
        _, s_m = sess.run([train, merged], {x: x_train, y: y_train})
        writer.add_summary(s_m, global_step=i)
    # evaluate training accuracy
    curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
    print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.999996] b: [0.99998856] loss: 2.2052582e-11


### 计算图

In [9]:
graph = tf.Graph()
with graph.as_default():
    # Model parameters
    W = tf.Variable([.3], dtype=tf.float32)
    b = tf.Variable([-.3], dtype=tf.float32)
    # Model input and output
    x = tf.placeholder(tf.float32)
    linear_model = W*x + b
    y = tf.placeholder(tf.float32)
    # loss
    loss = tf.reduce_mean(tf.square(linear_model - y)) # MSE
    summary_loss = tf.summary.scalar('MSE', loss)
    summary_b = tf.summary.scalar('b', b[0])
    # optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = optimizer.minimize(loss)

    
# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]

with tf.Session(graph=graph) as sess:
    # define summary writer
    writer = tf.summary.FileWriter('./tmp/visualization/', graph=graph)
    # merge summaries
    merged = tf.summary.merge_all()
    # training loop
    init = tf.global_variables_initializer()
    sess.run(init)
    for i in range(5000):
        _, s_m = sess.run([train, merged], {x: x_train, y: y_train})
        writer.add_summary(s_m, global_step=i)
    # evaluate training accuracy
    curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
    print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.999996] b: [0.99998856] loss: 2.2052582e-11


### 最终代码

In [10]:
graph = tf.Graph()
with graph.as_default():
    # Model parameters
    W = tf.Variable([.3], dtype=tf.float32, name='W')
    b = tf.Variable([-.3], dtype=tf.float32, name='b')
    # Model input and output
    x = tf.placeholder(tf.float32, name='input_x')
    y = tf.placeholder(tf.float32, name='input_y')
    with tf.name_scope('linear_model'):
        linear_model = W*x + b
    with tf.name_scope('cal_loss'):
        # loss
        loss = tf.reduce_mean(tf.square(linear_model - y), name='loss') # MSE
    with tf.name_scope('add_summary'):
        summary_loss = tf.summary.scalar('MSE', loss)
        summary_b = tf.summary.scalar('b', b[0])
    with tf.name_scope('train_model'):
        # optimizer
        optimizer = tf.train.GradientDescentOptimizer(0.01)
        train = optimizer.minimize(loss)

    
# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]

with tf.Session(graph=graph) as sess:
    # define summary writer
    writer = tf.summary.FileWriter('./tmp/visualization/', graph=graph)
    # merge summaries
    merged = tf.summary.merge_all()
    # training loop
    init = tf.global_variables_initializer()
    sess.run(init)
    for i in range(5000):
        _, s_m = sess.run([train, merged], {x: x_train, y: y_train})
        writer.add_summary(s_m, global_step=i)
    # evaluate training accuracy
    curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
    print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.999996] b: [0.99998856] loss: 2.2052582e-11


### 练习 ：以多项式回归的例子进行实验，增加模型保存以及可视化计算图的部分（10分钟）

In [11]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pickle
with open('./poly.pkl', 'rb') as f:
    x, y = pickle.load(f)
whole = np.array([x, y]).T
train = whole[:-64]
test = whole[-64:]
def gen_batch(data):
    for i in range(len(data) // 64):
        pos = 64 * i
        yield data[pos : pos + 64]
lr = 0.00001
epoch = 500


In [12]:
graph = tf.Graph()
with graph.as_default():
    # 定义placeholder
    x_t = tf.placeholder(shape=[None, 1], dtype=tf.float64, name='x')
    y_t = tf.placeholder(shape=[None, 1], dtype=tf.float64, name='y')
    
    # 计算x的n次方
    with tf.name_scope("compute_power_of_x"):
        x_pow = tf.concat([tf.ones(shape=tf.shape(x_t), dtype=tf.float64), 
                           x_t, tf.pow(x_t, 2), tf.pow(x_t, 3), tf.pow(x_t, 4)], axis=1)
        print(x_pow)
    # 定义权重
    W = tf.Variable(tf.truncated_normal(shape=(5, 1), dtype=tf.float64), name="weight_matrix")
    # 计算多项式
    y_pred = tf.matmul(x_pow, W)
    with tf.name_scope("calculate_loss"):
        # 定义loss
        loss = tf.reduce_mean(tf.pow(y_pred - y_t, 2))
    # 使用SGD对loss进行优化
    opt = tf.train.GradientDescentOptimizer(lr).minimize(loss)
    tf.summary.scalar('loss', loss)

Tensor("compute_power_of_x/concat:0", shape=(?, 5), dtype=float64)


In [ ]:
with tf.Session(graph=graph) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./tmp', graph=graph)
    saver = tf.train.Saver()
    step= 0
    for epoch in range(epoch):
        for minibatch in gen_batch(train):
            _, m = sess.run([opt, merged], 
                            feed_dict={x_t: np.reshape(minibatch[:, 0], (-1, 1)), 
                                       y_t: np.reshape(minibatch[:, 1], (-1, 1))})
            writer.add_summary(m, global_step=step)
            step += 1
    print("Training Finished")
    res, l = sess.run([W, loss], 
                            feed_dict={x_t: np.reshape(test[:, 0], (-1, 1)), 
                                       y_t: np.reshape(test[:, 1], (-1, 1))})
    print("Testing Loss:{:>11.4f}".format(l))
    saver.save(sess, './tmp/model.ckpt')

In [ ]:
with tf.Session(graph=graph) as sess:
    saver.restore(sess, './tmp/model.ckpt')
    res, l = sess.run([W, loss], 
                            feed_dict={x_t: np.reshape(test[:, 0], (-1, 1)), 
                                       y_t: np.reshape(test[:, 1], (-1, 1))})
    print("Testing Loss:{:>11.4f}".format(l))